# Qulacs チュートリアル(ミニマム）

In [8]:
import numpy as np
from qulacs import QuantumState

Qulacsは、pythonから利用することができ
pip instal qulacs
でインストールすることができる。実装は、C言語およびC++言語を使い、gccのopenMPを用いて高速化している。 このため、gcc-7, g++-7を必要とする。macユーザーの場合は、Xcodeからgccとg++を利用することができるが、デフォルトではclangが動いているので、brew等でgcc-7をインストールしておこう。qulacsのインストールの詳しい手続きは、
http://qulacs.org/md_1__how_to_install.html
を参考にしていただきたい。

### 量子ビットの準備
　$n$個の量子ビットの状態を表現するためには、$2^n$個の要素からなる複素ベクトルを準備する必要がある。量子状態に関数さまざまな関数は、`QuantumState`モジュールを用いて実行することができる。`QuantumState(n)`で量子ビット数nを指定して、状態を確保することができる。

In [9]:
nqubits = 3
state = QuantumState(nqubits)
print(state)

 *** Quantum State ***
 * Qubit Count : 3
 * Dimension   : 8
 * State vector : 
(1,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)



また、確保した量子ビットをすべて0状態、$|00...0\rangle$へと初期化は `set_zero_state()`で行うことができる。

量子状態を取得し、`array`として格納するには、`get_vector()`を使う。

In [10]:
state_array = state.get_vector()
print(state_array)

[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


ゼロ状態への初期化以外にも、ビット列を指定した初期化、`set_computational_basis()`、や、ランダムな量子状態の準備、`set_Haar_random_state()`、などの準備を行う関数も用意されている

In [11]:
state.set_computational_basis(0b101)
print(state.get_vector())

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]


逆に、arrayから量子状態を定義することもできる

In [12]:
import random
state_array = [random.random() for i in range(pow(2,nqubits))]
print(state_array)

[0.15613118728402808, 0.4829859977033609, 0.19285171584027738, 0.251401455014191, 0.40979077462637326, 0.26370421473684424, 0.05664163068464434, 0.6647642682373299]


In [13]:
state.load(state_array)
print(state.get_vector())

[0.15613119+0.j 0.482986  +0.j 0.19285172+0.j 0.25140146+0.j
 0.40979077+0.j 0.26370421+0.j 0.05664163+0.j 0.66476427+0.j]


この状態は規格化されていないが get_squared_norm() でノルムの２乗を取得して multiply_coef で定数倍することで規格化できる

In [14]:
norm_sqare = state.get_squared_norm()
print(norm_sqare)
state.multiply_coef(1.0/pow(norm_sqare,0.5))

1.0406350959966097


qulacsには他にも、量子状態のコピーや、

In [17]:
copied_state = state.copy()
print(copied_state.get_vector())

[0.1530525 +0.j 0.47346219+0.j 0.18904895+0.j 0.24644417+0.j
 0.40171028+0.j 0.25850434+0.j 0.05552474+0.j 0.65165605+0.j]


２つの量子状態の内積の計算、

In [18]:
from qulacs.state import inner_product
inner_product(copied_state,state)
#規格化されたベクトルの内積なので1になる。

(0.9999999999999999+0j)

量子状態のために確保したメモリの解放は、`del`から行うことができる。

In [52]:
del state
state

NameError: name 'state' is not defined

## 量子回路の構成

In [19]:
from qulacs.gate import Identity, X,Y,Z #パウリ演算子
from qulacs.gate import H,S,Sdag, sqrtX,sqrtXdag,sqrtY,sqrtYdag #1量子ビット Clifford演算
from qulacs.gate import T,Tdag #1量子ビット 非Clifford演算
from qulacs.gate import RX,RY,RZ #パウリ演算子についての回転演算
from qulacs.gate import CNOT, CZ, SWAP #2量子ビット演算

`Identity` $= \left(
\begin{array}{cc}
1 & 0
\\
0 & 1
\end{array}
\right)
$, 

`X` $= 
\left(
\begin{array}{cc}
0 & 1
\\
1 & 0
\end{array}
\right)
$, 


`Y` $
=
\left(
\begin{array}{cc}
0 & -i
\\
i & 0
\end{array}
\right)
$, 


`Z` $
=
\left(
\begin{array}{cc}
1 & 0
\\
0 & -1
\end{array}
\right)
$, 


`H` $
=
\frac{1}{\sqrt{2}}\left(
\begin{array}{cc}
1 & 1
\\
1 & -1
\end{array}
\right)
$, 


`S` $
=
\left(
\begin{array}{cc}
1 & 0
\\
0 & i
\end{array}
\right)
$,


`Sdag` $
=
\frac{1}{\sqrt{2}}\left(
\begin{array}{cc}
1 & 0
\\
0 & -i
\end{array}
\right)
$,


`sqrtX` $
=\sqrt{X}=
\frac{1}{2}\left(
\begin{array}{cc}
1+i & 1-i
\\
1-i & 1+i
\end{array}
\right)
$, 


`sqrtXdag` $
=\sqrt{X}^{\dagger}=
\frac{1}{2}\left(
\begin{array}{cc}
1-i & 1+i
\\
1+i & 1-i
\end{array}
\right)
$, 


`sqrtY` $
=\sqrt{Y}=
\frac{1}{2}\left(
\begin{array}{cc}
1+i & -1-i
\\
1+i & 1+i
\end{array}
\right)
$, 


`sqrtYdag` $
=\sqrt{Y}^{\dagger}=
\frac{1}{2}\left(
\begin{array}{cc}
1-i & -1+i
\\
1-i & 1-i
\end{array}
\right)
$

`T` 
$
= 
\left(
\begin{array}{cc}
1 & 0
\\
0 & e^{i \pi/4}
\end{array}
\right)
$, 

`Tdag` 
$=
\left(
\begin{array}{cc}
1 & 0
\\
0 & e^{-i \pi/4}
\end{array}
\right)
$

`RX` $= e^{i (\theta/2) X} = \cos (\theta/2) I + i \sin (\theta/2) X$, 

`RY` $= e^{i (\theta/2) Y} = \cos (\theta/2) I + i \sin (\theta/2) Y$, 

`RZ` $= e^{i (\theta/2) Z} = \cos (\theta/2) I + i \sin (\theta/2) Z$

`CNOT` $= \left(
\begin{array}{cccc}
    1 & 0& 0& 0
    \\
    0 & 1& 0& 0
    \\
    0 & 0& 0& 1
    \\
    0 & 0& 1& 0    
\end{array}
\right)
$,

`CZ` $= \left(
\begin{array}{cccc}
    1 & 0& 0& 0
    \\
    0 & 1& 0& 0
    \\
    0 & 0& 1& 0
    \\
    0 & 0& 0& -1    
\end{array}
\right)
$,

`SWAP` $= \left(
\begin{array}{cccc}
    1 & 0& 0& 0
    \\
    0 & 0& 1& 0
    \\
    0 & 1& 0& 0
    \\
    0 & 0& 0& 1    
\end{array}
\right)
$

他にも、多体のパウリ演算、`Pauli`、多体パウリ演算による回転、`PauliRotation`、IBMQで利用されている`U1`,`U2`,`U3`演算や、測定、`Measurement`、ノイズ（確率的な演算）、 `BitFlipNoise`、`DephasingNoise`、 `IndepenedentXZNoise`、`DepolarizingNoise`なども用意されている。これらの事前に定義された演算に対して、量子ビットのインデックスや、回転演算の角度など必要な情報を引数として指定することによってが`gate`が生成される。生成された演算を状態に作用させ状態`state`を更新するときは、`gate.update_quantum_state(state)`が利用できる。

In [20]:
nqubits = 1
state = QuantumState(nqubits)

H(0).update_quantum_state(state)
print(state.get_vector())
RZ(0,np.pi/2).update_quantum_state(state)
print(state.get_vector())

[0.70710678+0.j 0.70710678+0.j]
[0.5+0.5j 0.5-0.5j]


`DenseMatrix`を用いて、自分で定義した一般の行列から gate を作ることもできる。

In [82]:
from qulacs.gate import DenseMatrix

gate_array=[[random.random() for i in range(2)] for i in range(2)]
print(gate_array)

original_gate = DenseMatrix([0],gate_array)
state.set_zero_state()
print(state.get_vector())

original_gate.update_quantum_state(state)
print(state.get_vector())

[[0.8002537024306302, 0.5969354744062666], [0.8566878974192659, 0.749755777622155]]
[1.+0.j 0.+0.j]
[0.8002537+0.j 0.8566879+0.j]


複数の量子ビットに作用する演算を定義したければ、`DenseMatrix("作用する量子ビットのインデックスリスト","行列の配列")`とすればよい。

### 量子回路の構成

上で説明したように、各演算ごとに実行して量子状態を更新することもできるが、複雑な量子回路を構成する場合には、実行する前に量子回路を構成できると便利であろう。qulacsでは、`QuantumCircuit`を用いて、演算の集合からなる回路`circuit`を定義することができる。定義した回路をprintすると、構成された回路の詳細データを得ることができる。

In [21]:
from qulacs import QuantumCircuit
nqubits = 2
state = QuantumState(nqubits)
state.set_zero_state()
circuit = QuantumCircuit(nqubits)
print(circuit)

*** Quantum Circuit Info ***
# of qubit: 2
# of step : 0
# of gate : 0
Clifford  : yes
Gaussian  : yes




回路への、演算の追加は、以下のような方法で行うことができる。

In [22]:
circuit.add_H_gate(0) # 0番目の量子ビットに作用するHを追加
circuit.add_gate(H(1)) # 1番目の量子ビットに作用するHを追加
print(circuit)

*** Quantum Circuit Info ***
# of qubit: 2
# of step : 1
# of gate : 2
# of 1 qubit gate: 2
Clifford  : yes
Gaussian  : no




このようにして構成された`circuit`を実行して`state`の更新を行いたければ、`gate`の場合と同様に、
`circuit.update_quantum_state(state)`とすればよい。

In [23]:
circuit.update_quantum_state(state) #構成されたcircuitを実行して状態を更新
print(state.get_vector())

[0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]


### シミュレーションに要する時間

In [29]:
import time
nqubits = 28
state = QuantumState(nqubits)
circuit = QuantumCircuit(nqubits)

for i in range(nqubits):
    circuit.add_gate(H(i))

start_time = time.time()
circuit.update_quantum_state(state)
elapsed_time = time.time()-start_time
print(elapsed_time)

58.15091609954834


### サンプリングによる測定のシミュレーション

0,1基底に対するサンプリングは

In [30]:
nqubits = 20
state = QuantumState(nqubits)
state.set_Haar_random_state()

num_samp = 10
result = state.sampling(num_samp)
print(result)
print([bin(result[i]) for i in range(num_samp)])

[335240, 106179, 760972, 844662, 293016, 684303, 570992, 297198, 387366, 32365]
['0b1010001110110001000', '0b11001111011000011', '0b10111001110010001100', '0b11001110001101110110', '0b1000111100010011000', '0b10100111000100001111', '0b10001011011001110000', '0b1001000100011101110', '0b1011110100100100110', '0b111111001101101']


### 演算子の期待値の計算

パウリ演算子とその係数をつかって演算子を定義できる

In [31]:
from qulacs import Observable
nqubits = 5
coef = 2.0
# Set Pauli operators: X_0 X_1 Y_2 Z_4
Pauli_string = "X 0 X 1 Y 2 Z 4"
observable = Observable(nqubits)
observable.add_operator(coef,Pauli_string)

`get_expectation_value`を用いて期待値を計算できる

In [32]:
state = QuantumState(nqubits)
state.set_Haar_random_state() 
#ランダムな量子状態を準備
print(observable.get_expectation_value(state))

-0.0369391649365539
